In [ ]:
import numpy as np
import pickle

In [ ]:
P1 = 1  # player 1 plays
P2 = -1  # player 2 plays
N = 3 # NxN board
LEARNING_RATE = 0.5
GAMMA = 0.9
EXPLORE_RATE = 0.3
EXPLORE_DECAY = 0.9999

In [ ]:
class Game:
    def __init__(self, size, p1, p2):
        self.size = size
        self.board = np.zeros((self.size, self.size), dtype=int)
        self.board[0, :] = P1
        self.board[-1, :] = P2
        
        self.player1 = p1
        self.player2 = p2
        
        self.turns = 0
    
    def is_same(self, p, i, j, di, dj):
        """
        if all numbers in a row/col/diagonal are same
        """
        for k in range(self.size):
            if self.board[i][j] != p:
                return False
            i += di
            j += dj
        return True
    
    def is_winner(self, player):
        """
        return True if this player has won the game
        """
        # check rows
        if player == P2 and self.is_same(player, 0, 0, 0, 1):
            return True
        if player == P1 and self.is_same(player, 2, 0, 0, 1):
            return True
        if self.is_same(player, 1, 0, 0, 1):
            return True
        # check columns
        for j in range(self.size):
            if self.is_same(player, 0, j, 1, 0):
                return True
        # main diagonal
        if self.is_same(player, 0, 0, 1, 1):
            return True
        # secondary diagonal
        if self.is_same(player, 0, self.size-1, 1, -1):
            return True
        
        return False
    
    def is_draw(self):
        """
        returns True if the game has ended in draw
        """
        return self.turns >= 20000
            
    
    def get_winner(self):
        """
        return P1 if player 1 wins, P2 if player 2 wins, 0 if draw, None if not yet finished
        """
        if self.is_winner(P1):
            return P1
        elif self.is_winner(P2):
            return P2
        elif self.is_draw():
            return 0
        else:
            return None
        
    def get_rewards(self, winner):
        """
        return reward when game ends: (reward for first player, reward for second player)
        """
        if winner == P1:
            return (1, -1)
        elif winner == P2:
            return (-1, 1)
        else:
            return (-0.2, -0.1)   # player 2 gets more reward for draw
    
    def update(self, action, player):
        """
        update board according to player action
        """
        self.board[action[2]][action[3]] = player.player_idx
        self.board[action[0]][action[1]] = 0
        
    def reset(self):
        """
        reset game after finishing episode
        """
        self.board = np.zeros((self.size, self.size), dtype=int)
        self.board[0, :] = P1
        self.board[-1, :] = P2
        
    def state_action_hash(self, action, player):
        """
        get hash for (state, action) pair for Q function
        """
        board = self.board.copy()
        board[action[2]][action[3]] = player
        board[action[0]][action[1]] = 0
        return hash(str(board))
        
    def play_episode(self):
        while True:
            if self.turns % 2 == 0:
                action = self.player1.select_action(self)
                self.update(action, self.player1)
            else:
                action = self.player2.select_action(self)
                self.update(action, self.player2)
            winner = self.get_winner()
            if winner is not None:
                reward = self.get_rewards(winner)
                self.player1.propagate_return(reward[0])
                self.player2.propagate_return(reward[1])
                self.player1.reset()
                self.player2.reset()
                self.reset()
                print('Turns:', self.turns)
                break
            self.turns += 1
        return reward
    
    
    def play_episode_human(self):
        turn = 0
        while True:
            if turn % 2 == 0:
                print('AI turn!')
                action = self.player1.select_action(self)
                self.update(action, self.player1)
            else:
                print('Your turn!')
                action = self.player2.select_action(self)
                self.update(action, self.player2)
            self.print_board()
            winner = self.get_winner()
            if winner is not None:
                if winner == P1:
                    print('AI wins!')
                elif winner == P2:
                    print('You win!')
                else:
                    print('Draw!')
                break
            turn += 1
    
    def play_episode_bot(self):
        turn = 0
        while True:
            if turn % 2 == 0:
                print('Bot 1 turn!')
                action = self.player1.select_action(self)
                self.update(action, self.player1)
            else:
                print('Bot 2 turn!')
                action = self.player2.select_action(self)
                self.update(action, self.player2)
            self.print_board()
            winner = self.get_winner()
            if winner is not None:
                if winner == P1:
                    print('Bot 1 wins!')
                elif winner == P2:
                    print('Bot 2 wins!')
                else:
                    print('Draw!')
                break
            turn += 1
    
    def print_board(self):
        print('-------------')
        for i in range(self.size):
            s = '|'
            for j in range(self.size):
                if self.board[i][j] == P1:
                    token = 'x'
                elif self.board[i][j] == P2:
                    token = 'o'
                else:
                    token = '-'
                s += ' ' + token + ' |'
            print(s)
            print('-------------')
            
    
    def play(self, episodes):
        for i in range(episodes):
            print('--------------Episode {}------------'.format(i))
            print('Episode Reward:', self.play_episode())    

In [ ]:
class Player:
    def __init__(self, idx):
        self.states = []
        self.state_values = {}
        self.player_idx = idx
        self.learning_rate = LEARNING_RATE
        self.explore_rate = EXPLORE_RATE
        self.explore_decay = EXPLORE_DECAY
        self.gamma = GAMMA
        
        self.transitions = [(0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 2), (0, 1, 1, 1),
                           (0, 2, 0, 1), (0, 2, 1, 2), (0, 2, 1, 1), (1, 0, 1, 1), (1, 0, 0, 0), (1, 0, 2, 0), 
                           (1, 1, 1, 0), (1, 1, 1, 2), (1, 1, 0, 1), (1, 1, 2, 1), (1, 1, 0, 0), (1, 1, 0, 2), 
                           (1, 1, 2, 0), (1, 1, 2, 2), (1, 2, 1, 1), (1, 2, 0, 2), (1, 2, 2, 2), (2, 0, 1, 0), 
                           (2, 0, 2, 1), (2, 0, 1, 1), (2, 1, 2, 0), (2, 1, 2, 2), (2, 1, 1, 1), (2, 2, 2, 1), 
                           (2, 2, 1, 2), (2, 2, 1, 1)]
    
    def get_valid_moves(self, game):
        moves = []
#         for i in range(game.size):
#             for j in range(game.size):
#                 if game.board[i][j] == self.player_idx:
#                     for di in range(-1, 2):
#                         for dj in range(-1, 2):
#                             i1 = i + di
#                             j1 = j + dj
#                             if i1 >= 0 and j1 >= 0 and i1 < game.size and j1 < game.size and game.board[i1][j1] == 0:
#                                 moves.append((i, j, i1, j1))
        for x in self.transitions:
            i = x[0]
            j = x[1]
            i1 = x[2]
            j1 = x[3]
            if game.board[i][j] == self.player_idx and game.board[i1][j1] == 0:
                moves.append((i, j, i1, j1))
        
        
        return moves
    
    def select_action(self, game):
        """
        Select action given a Game state
        """
        options = self.get_valid_moves(game)
        # print('options:', options)
        action = None
        if np.random.uniform(0, 1) <= self.explore_rate:
            # Exploration
            action = options[np.random.choice(len(options))]
        else:
            # Exploitation
            max_reward = -1e8
            for a in options:
                h = game.state_action_hash(a, self.player_idx)
                if h in self.state_values:
                    reward = self.state_values[h]
                else:
                    reward = 0
                if reward >= max_reward:
                    max_reward = reward
                    action = a
        # print('action:', action)
        self.states.append(game.state_action_hash(action, self.player_idx))
        return action 
    
    def propagate_return(self, reward):
        """
        back-propagate the returns for state-actions visited during this episode
        """
        for s in reversed(self.states):
            if s not in self.state_values:
                self.state_values[s] = 0
            self.state_values[s] += self.learning_rate * (reward - self.state_values[s])
            # reward *= self.gamma
    
    def reset(self):
        """
        reset states and lower explore rate
        """
        self.states = []
        # self.explore_rate *= self.explore_decay
        
    def save_policy(self, fname):
        """
        store policy
        """
        with open(fname, 'wb') as handle:
            pickle.dump(self.state_values, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    def load_policy(self, fname):
        """
        load policy
        """
        with open(fname, 'rb') as handle:
            self.state_values = pickle.load(handle)       

In [ ]:
class HumanPlayer:
    def __init__(self, idx):
        self.player_idx = idx
        
        self.transitions = [(0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 2), (0, 1, 1, 1),
                           (0, 2, 0, 1), (0, 2, 1, 2), (0, 2, 1, 1), (1, 0, 1, 1), (1, 0, 0, 0), (1, 0, 2, 0), 
                           (1, 1, 1, 0), (1, 1, 1, 2), (1, 1, 0, 1), (1, 1, 2, 1), (1, 1, 0, 0), (1, 1, 0, 2), 
                           (1, 1, 2, 0), (1, 1, 2, 2), (1, 2, 1, 1), (1, 2, 0, 2), (1, 2, 2, 2), (2, 0, 1, 0), 
                           (2, 0, 2, 1), (2, 0, 1, 1), (2, 1, 2, 0), (2, 1, 2, 2), (2, 1, 1, 1), (2, 2, 2, 1), 
                           (2, 2, 1, 2), (2, 2, 1, 1)]
        
    def get_valid_moves(self, game):
        moves = []
#         for i in range(game.size):
#             for j in range(game.size):
#                 if game.board[i][j] == self.player_idx:
#                     for di in range(-1, 2):
#                         for dj in range(-1, 2):
#                             i1 = i + di
#                             j1 = j + dj
#                             if i1 >= 0 and j1 >= 0 and i1 < game.size and j1 < game.size and game.board[i1][j1] == 0:
#                                 moves.append((i, j, i1, j1))
        for x in self.transitions:
            i = x[0]
            j = x[1]
            i1 = x[2]
            j1 = x[3]
            if game.board[i][j] == self.player_idx and game.board[i1][j1] == 0:
                moves.append((i, j, i1, j1))
        
        
        return moves
    
    def select_action(self, game):
        """
        Select action given a Game state
        """
        options = self.get_valid_moves(game)
        while True:
            from_row = int(input('From row:'))
            from_col = int(input('From col:'))
            to_row = int(input('To row:'))
            to_col = int(input('To col:'))
            if (from_row, from_col, to_row, to_col) in options:
                return (from_row, from_col, to_row, to_col)
            else:
                print('Invalid row/column!')

In [ ]:
p1 = Player(P1)
p2 = Player(P2)
game = Game(N, p1, p2)
game.play(50000)
p1.save_policy('tinguti_p1.pickle')
p2.save_policy('tinguti_p2.pickle')

In [ ]:
p1 = Player(P1)
p1.load_policy('tinguti_p1.pickle')
p1.explore_rate = 0

p2 = HumanPlayer(P2)
game = Game(N, p1, p2)
game.play_episode_human()

In [ ]:
p1 = Player(P1)
p1.load_policy('tinguti_p1.pickle')
p1.explore_rate = 0

p2 = Player(P2)
p2.load_policy('tinguti_p2.pickle')
p2.explore_rate = 0

game = Game(N, p1, p2)
game.play_episode_bot()